In [ ]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import pandas as pd 
import numpy as np 
import sqlite3 as sql
import seaborn as sn
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score 
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)
pd.set_option('display.width', 500)








In [ ]:
# %%
db = sql.connect('../data/combinedLAS_v2.db')

#%%
dfa = pd.read_sql('SELECT * FROM all_LAS',db)
dfa.tail()

# %%
# Load selected features from SQL into dataframe
dfs = pd.read_sql('SELECT TVDSS,WELLNAME,GR,RTC,NPHIC,RHOBC,SWT,FLUID,KLOG,PHIT_HC FROM all_LAS\
                            WHERE GR IS NOT NULL\
                            AND RTC IS NOT NULL\
                            AND NPHIC IS NOT NULL\
                            AND RHOBC IS NOT NULL\
                            AND TVDSS IS NOT NULL\
                            AND SWT IS NOT NULL\
                            AND KLOG IS NOT NULL\
                            AND PHIT_HC IS NOT NULL\
                            AND FLUID IS NOT NULL', db)#, index_col='index')
print(dfs.shape)
dfs.head()
#df.to_sql('selected_LAS', db, if_exists='replace')

#%%
df = pd.read_sql('SELECT * FROM cleaned_LAS',db,index_col='index')
df.head()

#%%
#dfc.to_sql('cleaned_LAS', db, if_exists='replace')
db.close()


In [ ]:
# %%
# Merge similar log types on all_LAS dataframe

#dfa[['DEN','RHOB']].describe()
rRHOB = dfa[(dfa.DEN.isna() | (dfa.DEN <0))& dfa.RHOB.notna()][['DEN','RHOB']]
rNPHI = dfa[(dfa.NEUT.isna() | dfa.NEUT<-0.02)& dfa.NPHI_COR.notna()][['NEUT','NPHI_COR']]
rRT = dfa[dfa.LLD.isna() & dfa.RT.notna()][['LLD','RT']]
print('%s \n\n%s \n\n%s' % (rRHOB.count(), rNPHI.count(), rRT.count()))

dfa['RHOBC'] = np.where(dfa.DEN.isna(), dfa.RHOB, dfa.DEN)
dfa['RHOBC'] = np.where(dfa.RHOBC<0, dfa.RHOB, dfa.RHOBC)

dfa['NPHIC'] = np.where(dfa.NEUT.isna(), dfa.NPHI_COR, dfa.NEUT)
dfa['NPHIC'] = np.where(dfa.NPHIC<-0.02, dfa.NPHI_COR, dfa.NPHIC)
dfa['NPHIC'] = np.where(dfa.NPHIC>=1, dfa.NPHIC/100, dfa.NPHIC)
dfa['NPHIC'] = np.where(dfa.NPHIC>=1, np.nan, dfa.NPHIC)

dfa['RTC'] = np.where(dfa.LLD.isna(), dfa.RT, dfa.LLD)
dfa['RTC'] = np.where(dfa.RTC<0, np.nan, dfa.RTC)
print('\nMissing values (pct): \n %s' % round(dfa.isna().sum()/dfa.shape[0]*100,2).sort_values())


#%% 
# Data QC
print('Dataframe shape: %d, %d' % dfs.shape)
print('\nMissing values, (pct): \n %s' % round(dfs.isna().sum()/dfs.shape[0]*100,2).sort_values())
print('\nDescibe(); \n %s' % round(dfs.describe(),2))
print('\nKurtosis(); \n %s' % round(dfs.kurt()))
print('\nSkew(); \n %s' % round(dfs.skew()))
dfs.hist(figsize=(10,10), bins=20)


#%% 
# Data QC
print('Dataframe shape: %d, %d' % df.shape)
print('\nMissing values, (pct): \n %s' % round(df.isna().sum()/df.shape[0]*100,2).sort_values())
print('\nDescibe(); \n %s' % round(df.describe(),2))
print('\nKurtosis(); \n %s' % round(df.kurt()))
print('\nSkew(); \n %s' % round(df.skew()))
df.hist(figsize=(10,10), bins=20)

# %%
# List missing data by Well
print('\nNumber  of rows by well: ')
g = df.groupby('WELLNAME')
#g.count().rsub(g.size(), axis=0)
#g.count().rsub(g.size(), axis=0).sum(axis=1).sort_values()
g.count().sum(axis=1).sort_values(ascending=False)


#%%
# Clean out of range data/ features
dfc=df
print(round(dfc.describe(),2))

dfc.GR = np.where(dfc.GR>250,np.nan,dfc.GR)
dfc.RTC = np.where(dfc.RTC>200,np.nan,dfc.RTC)
dfc.NPHIC = np.where(dfc.NPHIC>.5,np.nan,dfc.NPHIC)
dfc.RHOBC = np.where(dfc.RHOBC>3,np.nan,dfc.RHOBC)
dfc.RHOBC = np.where(dfc.RHOBC<1.1,np.nan,dfc.RHOBC)
dfc.KLOG = np.where(dfc.KLOG>2000,2000,dfc.KLOG)
dfc.PHIT_HC = np.where(dfc.PHIT_HC>.5,np.nan,dfc.PHIT_HC)

dfc.dropna(inplace=True)
print(round(dfc.describe(),2))





In [ ]:
#%%
# Normalize data and check correlation
dfn = df
df_norm = (
    dfn.groupby("WELLNAME")
    .apply(SklearnWrapper(preprocessing.MinMaxScaler()))#StandardScaler()))
    #.drop("WEclass", axis="columns")
)
df_norm['WELLNAME'] = dfn['WELLNAME']
print(df_norm.columns)
print(round(df_norm.describe(),2))

print('\nCorrelation for all wells (normalized)')
dfcorr = df_norm.corr(method='kendall')
sn.heatmap(dfcorr, annot=True,fmt='.2f')
plt.show()


#%%
dfn45 = df_norm[df_norm['WELLNAME']=='WELL-45']
dfn45.describe()


In [ ]:
# %%
print('Modelling regressors on SWT for Well-45')

""" X = df_norm.iloc[:,0:5]
y = df_norm.iloc[:,5] # Target variable """
X = dfn45.iloc[:,0:5]
y = dfn45.iloc[:,5] # Target variable

X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.20, random_state=1)

print('Target variable: SWT, Features: TVDSS, GR, RTC, NPHIC, RHOBC') 
# SWT
compareRModels()



#%%
# Tuning SVR parameters for estimator
param = {'kernel':('poly', 'sigmoid','rbf'), 'C':[1, 2, 3]}
GS_CV = GridSearchCV(SVR(),param)
GS_CV.fit(X_train, Y_train)
print(GS_CV.get_params())
#est.predict(X_validation)
print(GS_CV.score(X_validation,Y_validation))
print(GS_CV.best_params_)

#%%
est = SVR(C=3,kernel='poly')
est.fit(X_train, Y_train)
est.score(X_validation,Y_validation)


In [ ]:
#%%
def compareCModels():#X_train, Y_train):
    # Spot Check Algorithms
    models = []
    models.append(('LR', LogisticRegression()))
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('KNNC', KNeighborsClassifier()))    
    models.append(('CART', DecisionTreeClassifier()))    
    models.append(('NB', GaussianNB()))        
    models.append(('SVC', SVC(gamma='auto')))   
    # evaluate each model in turn
    results = []
    names = []
    mean_results = []
    for name, model in models:
        kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
        cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy') #cv=kfold
        results.append(cv_results)
        names.append(name)
        mean_results.append(cv_results.mean())
        print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
    plt.boxplot(results, labels=names)
    plt.title('Algorithm Comparison')
    plt.show()

    bestModel = models[np.nanargmax(mean_results)][1]
    print('\nBest model: %s' % (bestModel))
    modelScore(bestModel)
    return bestModel

def compareRModels():#X_train, Y_train):
    # Spot Check Algorithms
    models = []    
    models.append(('KNNR', KNeighborsRegressor()))    
    models.append(('DTR', DecisionTreeRegressor()))
    models.append(('ADAB', AdaBoostRegressor()))       
    models.append(('GPR', GaussianProcessRegressor()))    
    #models.append(('SVR', SVR(gamma='auto')))
    models.append(('MLPR', MLPRegressor(random_state=1)))
    # evaluate each model in turn
    results = []
    names = []
    mean_results = []
    for name, model in models:
        cv_results = cross_val_score(model, X_train, Y_train, scoring='r2')
        results.append(cv_results)
        names.append(name)
        mean_results.append(cv_results.mean())
        print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
    plt.boxplot(results, labels=names)
    plt.title('Algorithm Comparison')
    plt.show()

    bestModel = models[np.nanargmax(mean_results)][1]
    print('\nBest model: %s' % (bestModel))
    modelScore(bestModel)


def modelScore(model):
    #model = GaussianNB()
    model.fit(X_train, Y_train)
    predictions = model.predict(X_validation)

    # Evaluate predictions
    try:
        print('Accuracy score: %f' % (accuracy_score(Y_validation, predictions)))
        #print(confusion_matrix(Y_validation, predictions))
        print('Classification report: \n %s' % (classification_report(Y_validation, predictions)))
    except:
        print('R2: %f' % (model.score(X_train, Y_train)))
        #print(confusion_matrix(Y_validation, predictions))
        #print('Classification report: \n %s' % 

import typing
class SklearnWrapper:
    def __init__(self, transform: typing.Callable):
        self.transform = transform

    def __call__(self, df):
        transformed = self.transform.fit_transform(df.values)
        return pd.DataFrame(transformed, columns=df.columns, index=df.index)